# Базовая подготовка наборов данных (частично обработаны в другом файле)

### Импорт библиотек

In [3]:
import pandas as pd
import requests
import time
from functools import reduce

#### IMOEX

In [5]:
imoex_df = pd.read_excel("imoex_data_2022_2025.xlsx")

imoex_df = imoex_df.rename(columns={
    'Дата': 'date',
    'Цена': 'price',
    'Откр.': 'open',
    'Макс.': 'high',
    'Мин.': 'low',
    'Объём': 'volume',
    'Изм. %': 'change_percent'
})

imoex_df = imoex_df.rename(columns=lambda x: f"imoex_{x}" if x != 'date' else x)

In [6]:
imoex_df

,date,imoex_price,imoex_open,imoex_high,imoex_low,imoex_volume,imoex_change_percent
0,2025-04-01,"2.964,43","3.032,41","3.044,23","2.952,04","87,84B",-0.0162
1,2025-03-31,"3.013,36","3.008,60","3.031,60","2.970,94","77,26B",-0.0041
2,2025-03-28,"3.025,65","3.055,81","3.082,26","3.001,62","106,84B",-0.0189
3,2025-03-27,"3.083,99","3.120,47","3.124,32","3.076,04","76,69B",-0.0198
4,2025-03-26,"3.146,31","3.195,03","3.200,02","3.143,65","47,01B",-0.0058
...,...,...,...,...,...,...,...
757,2022-04-07,"2.635,39","2.618,26","2.653,85","2.618,26",NaN,0.0092
758,2022-04-06,"2.611,43","2.641,79","2.696,97","2.594,95",NaN,-0.0193
759,2022-04-05,"2.662,79","2.796,56","2.808,27","2.576,34",NaN,-0.0448
760,2022-04-04,"2.787,69","2.775,06","2.835,46","2.682,34",NaN,0.0102


#### Ставка Центрального Банка РФ (ЦБ РФ)

In [8]:
# https://www.cbr.ru/hd_base/keyrate/ - отсюда

In [9]:
cb_df = pd.read_excel("cb_rate_data.xlsx")
cb_df.head()

,date,cb_rate
0,2025-04-01,0.21
1,2025-03-31,0.21
2,2025-03-28,0.21
3,2025-03-27,0.21
4,2025-03-26,0.21


#### ИНДЕКС ДОЛЛАРА

In [11]:
# https://ru.investing.com/indices/usdollar-historical-data

# Загрузка файла
dollar_index_df = pd.read_excel("USD_index_data.xlsx")

# Переименование столбцов с русского на английский
dollar_index_df = dollar_index_df.rename(columns={
    'Дата': 'date',
    'Цена': 'price',
    'Откр.': 'open',
    'Макс.': 'high',
    'Мин.': 'low',
    'Объём': 'volume',
    'Изм. %': 'change_percent'
})

# Добавление префикса dxy_ ко всем столбцам, кроме 'date'
dollar_index_df = dollar_index_df.rename(columns=lambda x: f"dxy_{x}" if x != 'date' else x)

# Просмотр первых строк
dollar_index_df.head()

,date,dxy_price,dxy_open,dxy_high,dxy_low,dxy_volume,dxy_change_percent
0,2025-04-01,104.26,104.18,104.37,104.02,"0,00K",0.0005
1,2025-03-31,104.21,104.01,104.39,103.75,"0,00K",0.0016
2,2025-03-28,104.04,104.28,104.50,103.90,"0,00K",-0.0028
3,2025-03-27,104.33,104.65,104.65,104.07,"0,00K",-0.0020
4,2025-03-26,104.55,104.25,104.68,104.18,"0,00K",0.0035


#### Цены на золото

In [13]:
# https://investfunds.ru/indexes/224/

In [14]:
gold_df = pd.read_excel("gold_prices.xlsx")

In [15]:
gold_df.head()

,date,gold_price
0,2025-04-01,8443.12
1,2025-03-31,8223.39
2,2025-03-30,8223.39
3,2025-03-29,8223.39
4,2025-03-28,8121.75


#### Индексы нефти

In [17]:
# https://fred.stlouisfed.org/series/DCOILWTICO
# https://fred.stlouisfed.org/series/DCOILBRENTEU

In [18]:
DCOILWTICO_df = pd.read_excel("DCOILWTICO.xlsx")
DCOILBRENTEU_df = pd.read_excel("DCOILBRENTEU.xlsx")

DCOILWTICO_df['west_price'] = DCOILWTICO_df['DCOILWTICO_price']
DCOILWTICO_df = DCOILWTICO_df.drop('DCOILWTICO_price', axis=1)
DCOILBRENTEU_df['brent_price'] = DCOILBRENTEU_df['DCOILBRENTEU_price']
DCOILBRENTEU_df = DCOILBRENTEU_df.drop('DCOILBRENTEU_price', axis=1)

In [19]:
DCOILWTICO_df.head()

,date,west_price
0,2022-04-01,99.32
1,2022-04-04,103.29
2,2022-04-05,101.98
3,2022-04-06,96.39
4,2022-04-07,96.05


In [20]:
DCOILBRENTEU_df.head()

,date,brent_price
0,2022-04-01,106.13
1,2022-04-04,108.15
2,2022-04-05,106.60
3,2022-04-06,100.81
4,2022-04-07,99.83


### Курсы валют

In [22]:
# https://www.investing.com/currencies/eur-rub-historical-data
# https://www.investing.com/currencies/usd-rub-historical-data

#### EUR_RUB

In [24]:
EUR_RUB_df = pd.read_excel("EUR_RUB_data.xlsx")

EUR_RUB_df = EUR_RUB_df.rename(columns={
    'Дата': 'date',
    'Цена': 'price',
    'Откр.': 'open',
    'Макс.': 'high',
    'Мин.': 'low',
    'Объём': 'volume',
    'Изм. %': 'change_percent'
})

EUR_RUB_df = EUR_RUB_df.rename(columns=lambda x: f"eur_rub_{x}" if x != 'date' else x)
EUR_RUB_df.head()

,date,eur_rub_price,eur_rub_open,eur_rub_high,eur_rub_low,eur_rub_volume,eur_rub_change_percent
0,2025-04-01,91.196,89.785,92.121,89.756,NaN,0.0158
1,2025-03-31,89.776,91.933,93.658,89.702,NaN,-0.0239
2,2025-03-28,91.970,91.273,92.826,89.916,NaN,0.0077
3,2025-03-27,91.264,90.410,91.770,89.892,NaN,0.0096
4,2025-03-26,90.393,91.319,91.378,90.165,NaN,-0.0101


#### USD_RUB

In [26]:
USD_RUB_df = pd.read_excel("USD_RUB_data.xlsx")

USD_RUB_df = USD_RUB_df.rename(columns={
    'Дата': 'date',
    'Цена': 'price',
    'Откр.': 'open',
    'Макс.': 'high',
    'Мин.': 'low',
    'Объём': 'volume',
    'Изм. %': 'change_percent'
})


USD_RUB_df = USD_RUB_df.rename(columns=lambda x: f"usd_rub_{x}" if x != 'date' else x)
USD_RUB_df.head()

,date,usd_rub_price,usd_rub_open,usd_rub_high,usd_rub_low,usd_rub_volume,usd_rub_change_percent
0,2025-04-01,84.4955,84.51,85.25,84.325,NaN,0.0181
1,2025-03-31,82.9955,83.86,86.50,83.000,NaN,-0.0230
2,2025-03-28,84.9455,83.56,85.70,83.375,NaN,0.0053
3,2025-03-27,84.4955,83.96,85.00,83.500,NaN,0.0051
4,2025-03-26,84.0705,83.66,84.65,83.660,NaN,-0.0065


### ИНДЕКСЫ

In [28]:
def fetch_moex_index_history(index_code, start_date='2022-04-01', end_date='2025-04-01', sleep_time=0.5):
    base_url = f"https://iss.moex.com/iss/history/engines/stock/markets/index/securities/{index_code}.json"
    start = 0
    all_data = []

    while True:
        params = {
            'from': start_date,
            'till': end_date,
            'start': start
        }

        try:
            response = requests.get(base_url, params=params, timeout=10)
            response.raise_for_status()
            data = response.json()
        except requests.exceptions.RequestException as e:
            print(f"Ошибка запроса: {e}")
            time.sleep(5)
            continue

        history = data.get('history', {}).get('data', [])
        columns = data.get('history', {}).get('columns', [])

        if not history:
            break

        all_data.extend(history)
        start += len(history)
        time.sleep(sleep_time)

    if not all_data:
        print(f"Нет данных для индекса {index_code}")
        return pd.DataFrame()

    df = pd.DataFrame(all_data, columns=columns)
    df = df[['TRADEDATE', 'OPEN', 'HIGH', 'LOW', 'CLOSE', 'VOLUME', 'VALUE']]
    df.rename(columns={
        'TRADEDATE': 'date',
        'OPEN': 'open',
        'HIGH': 'high',
        'LOW': 'low',
        'CLOSE': 'close',
        'VOLUME': 'volume',
        'VALUE': 'value'
    }, inplace=True)

    df['date'] = pd.to_datetime(df['date'])
    df.sort_values('date', inplace=True)
    df.reset_index(drop=True, inplace=True)

    return df

In [29]:
moexog_df = fetch_moex_index_history('MOEXOG')   # Нефти и газа

In [30]:
moexeu_df = fetch_moex_index_history('MOEXEU')   # Электроэнергетики

In [31]:
moextl_df = fetch_moex_index_history('MOEXTL')   # Телекоммуникаций

In [32]:
moexmm_df = fetch_moex_index_history('MOEXMM')   # Металлов и добычи

In [33]:
moexfn_df = fetch_moex_index_history('MOEXFN')   # Финансов

In [34]:
moexcn_df = fetch_moex_index_history('MOEXCN')   # Потребительского сектора

In [35]:
moexch_df = fetch_moex_index_history('MOEXCH')   # Химии и нефтехимии

In [36]:
moexit_df = fetch_moex_index_history('MOEXIT')   # Информационных технологий

In [37]:
moexre_df = fetch_moex_index_history('MOEXRE')   # Строительных компаний

In [38]:
moextn_df = fetch_moex_index_history('MOEXTN')   # Транспорта

In [39]:
# Переименование столбцов с добавлением префикса (кроме 'date')
moexog_df = moexog_df.rename(columns=lambda x: f"moexog_{x}" if x != 'date' else x)
moexeu_df = moexeu_df.rename(columns=lambda x: f"moexeu_{x}" if x != 'date' else x)
moextl_df = moextl_df.rename(columns=lambda x: f"moextl_{x}" if x != 'date' else x)
moexmm_df = moexmm_df.rename(columns=lambda x: f"moexmm_{x}" if x != 'date' else x)
moexfn_df = moexfn_df.rename(columns=lambda x: f"moexfn_{x}" if x != 'date' else x)
moexcn_df = moexcn_df.rename(columns=lambda x: f"moexcn_{x}" if x != 'date' else x)
moexch_df = moexch_df.rename(columns=lambda x: f"moexch_{x}" if x != 'date' else x)
moexit_df = moexit_df.rename(columns=lambda x: f"moexit_{x}" if x != 'date' else x)
moexre_df = moexre_df.rename(columns=lambda x: f"moexre_{x}" if x != 'date' else x)
moextn_df = moextn_df.rename(columns=lambda x: f"moextn_{x}" if x != 'date' else x)

In [40]:
moextn_df.head()

,date,moextn_open,moextn_high,moextn_low,moextn_close,moextn_volume,moextn_value
0,2022-04-01,976.16,1033.88,976.16,1003.18,None,1.447783e+09
1,2022-04-04,1027.17,1043.45,979.70,1033.29,None,2.443441e+09
2,2022-04-05,1040.41,1040.41,968.48,984.29,None,1.145121e+09
3,2022-04-06,971.09,1028.21,949.27,991.44,None,8.706356e+08
4,2022-04-07,995.42,1029.83,990.26,1007.51,None,6.927625e+08


In [41]:
VIX_raw_df = pd.read_excel("VIX_raw.xlsx")
DJIA_raw_df = pd.read_excel("DJIA_raw.xlsx")
RVI_raw_df = pd.read_excel("RVI_raw.xlsx")
S_P_500_raw_df = pd.read_excel("S_P_500_raw.xlsx")
NASDAQ_raw_data_df = pd.read_excel("NASDAQ_raw_data.xlsx")

In [42]:
# VIX
VIX_raw_df = VIX_raw_df.rename(columns={
    'Дата': 'date',
    'Цена': 'price',
    'Откр.': 'open',
    'Макс.': 'high',
    'Мин.': 'low',
    'Объём': 'volume',
    'Изм. %': 'change_percent'
})
VIX_raw_df = VIX_raw_df.rename(columns=lambda x: f"vix_{x}" if x != 'date' else x)

# DJIA
DJIA_raw_df = DJIA_raw_df.rename(columns={
    'Дата': 'date',
    'Цена': 'price',
    'Откр.': 'open',
    'Макс.': 'high',
    'Мин.': 'low',
    'Объём': 'volume',
    'Изм. %': 'change_percent'
})
DJIA_raw_df = DJIA_raw_df.rename(columns=lambda x: f"djia_{x}" if x != 'date' else x)

# RVI
RVI_raw_df = RVI_raw_df.rename(columns={
    'Дата': 'date',
    'Цена': 'price',
    'Откр.': 'open',
    'Макс.': 'high',
    'Мин.': 'low',
    'Объём': 'volume',
    'Изм. %': 'change_percent'
})
RVI_raw_df = RVI_raw_df.rename(columns=lambda x: f"rvi_{x}" if x != 'date' else x)

# S&P 500
S_P_500_raw_df = S_P_500_raw_df.rename(columns={
    'Дата': 'date',
    'Цена': 'price',
    'Откр.': 'open',
    'Макс.': 'high',
    'Мин.': 'low',
    'Объём': 'volume',
    'Изм. %': 'change_percent'
})
S_P_500_raw_df = S_P_500_raw_df.rename(columns=lambda x: f"sp500_{x}" if x != 'date' else x)

# NASDAQ
NASDAQ_raw_data_df = NASDAQ_raw_data_df.rename(columns={
    'Дата': 'date',
    'Цена': 'price',
    'Откр.': 'open',
    'Макс.': 'high',
    'Мин.': 'low',
    'Объём': 'volume',
    'Изм. %': 'change_percent'
})
NASDAQ_raw_data_df = NASDAQ_raw_data_df.rename(columns=lambda x: f"nasdaq_{x}" if x != 'date' else x)

In [43]:
S_P_500_raw_df.head()

,date,sp500_price,sp500_open,sp500_high,sp500_low,sp500_volume,sp500_change_percent
0,2025-04-01,"5.633,07","5.597,53","5.650,57","5.558,52",NaN,0.0038
1,2025-03-31,"5.611,85","5.527,91","5.627,56","5.488,73",NaN,0.0055
2,2025-03-28,"5.580,94","5.679,20","5.685,89","5.572,42",NaN,-0.0197
3,2025-03-27,"5.693,31","5.695,64","5.732,28","5.670,94",NaN,-0.0033
4,2025-03-26,"5.712,20","5.771,66","5.783,62","5.694,41",NaN,-0.0112


### Начинаем собирать данные в один датафрейм

#### Преобразуем даты в один формат

In [46]:
# Список всех датафреймов
dataframes = [
    imoex_df, cb_df, dollar_index_df, gold_df, DCOILWTICO_df,
    DCOILBRENTEU_df, EUR_RUB_df, USD_RUB_df, moexog_df, moexeu_df,
    moextl_df, moexmm_df, moexfn_df, moexcn_df, moexch_df,
    moexit_df, moexre_df, moextn_df, VIX_raw_df, DJIA_raw_df,
    RVI_raw_df, S_P_500_raw_df, NASDAQ_raw_data_df
]

# Приведение столбца date к типу datetime
for df in dataframes:
    if 'date' in df.columns:
        df['date'] = pd.to_datetime(df['date'], errors='coerce', dayfirst=True)

#### Объединение

In [48]:
# Список всех датафреймов
dataframes = [
    imoex_df, cb_df, dollar_index_df, gold_df, DCOILWTICO_df,
    DCOILBRENTEU_df, EUR_RUB_df, USD_RUB_df, moexog_df, moexeu_df,
    moextl_df, moexmm_df, moexfn_df, moexcn_df, moexch_df,
    moexit_df, moexre_df, moextn_df, VIX_raw_df, DJIA_raw_df,
    RVI_raw_df, S_P_500_raw_df, NASDAQ_raw_data_df
]

# Выделяем базовый датафрейм
base_df = dataframes[0]

# Объединяем остальные датафреймы к базовому по колонке 'date'
data_df = reduce(
    lambda left, right: pd.merge(left, right, on='date', how='left'),
    dataframes[1:],  # Все, кроме base_df
    base_df          # Начинаем с imoex_df
)

# Сортировка по дате и сброс индекса
data_df = data_df.sort_values('date').reset_index(drop=True)

In [49]:
data_df['eur_rub_price'].isna().sum()

3

#### Удалим объемы

In [51]:
# Удаляем все колонки, в названии которых содержится 'volume'
data_df = data_df.drop(columns=[col for col in data_df.columns if 'volume' in col])

In [52]:
initial_data = data_df

In [53]:
def convert_european_numbers(df):
    for col in df.columns:
        if df[col].dtype == 'object':
            df[col] = df[col].str.replace('.', '', regex=False)  # убираем точки из тысяч
            df[col] = df[col].str.replace(',', '.', regex=False)  # заменяем запятые на точки
            try:
                df[col] = df[col].astype(float)
            except ValueError:
                pass  # если не получилось - оставляем строкой
    return df

In [54]:
convert_european_numbers(data_df)

,date,imoex_price,imoex_open,imoex_high,imoex_low,imoex_change_percent,cb_rate,dxy_price,dxy_open,dxy_high,...,sp500_price,sp500_open,sp500_high,sp500_low,sp500_change_percent,nasdaq_price,nasdaq_open,nasdaq_high,nasdaq_low,nasdaq_change_percent
0,2022-04-01,2759.64,2714.62,2817.72,2714.62,0.0208,0.20,98.63,98.33,98.74,...,4545.86,4540.32,4548.70,4507.57,0.0034,14261.50,14269.53,14306.94,14131.81,0.0029
1,2022-04-04,2787.69,2775.06,2835.46,2682.34,0.0102,0.20,99.00,98.57,99.08,...,4582.64,4547.97,4583.50,4539.21,0.0081,14532.55,14304.35,14534.38,14286.45,0.0190
2,2022-04-05,2662.79,2796.56,2808.27,2576.34,-0.0448,0.20,99.47,99.00,99.52,...,4525.12,4572.45,4593.45,4514.17,-0.0126,14204.17,14490.26,14500.29,14169.12,-0.0226
3,2022-04-06,2611.43,2641.79,2696.97,2594.95,-0.0193,0.20,99.60,99.47,99.77,...,4481.15,4494.17,4503.94,4450.04,-0.0097,13888.82,14002.58,14032.84,13788.90,-0.0222
4,2022-04-07,2635.39,2618.26,2653.85,2618.26,0.0092,0.20,99.75,99.65,99.83,...,4500.21,4474.65,4521.16,4450.30,0.0043,13897.30,13861.49,13978.25,13689.23,0.0006
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
757,2025-03-26,3146.31,3195.03,3200.02,3143.65,-0.0058,0.21,104.55,104.25,104.68,...,5712.20,5771.66,5783.62,5694.41,-0.0112,17899.02,18217.33,18236.55,17837.26,-0.0204
758,2025-03-27,3083.99,3120.47,3124.32,3076.04,-0.0198,0.21,104.33,104.65,104.65,...,5693.31,5695.64,5732.28,5670.94,-0.0033,17804.03,17811.24,17988.02,17743.80,-0.0053
759,2025-03-28,3025.65,3055.81,3082.26,3001.62,-0.0189,0.21,104.04,104.28,104.50,...,5580.94,5679.20,5685.89,5572.42,-0.0197,17322.99,17722.09,17763.28,17283.06,-0.0270
760,2025-03-31,3013.36,3008.60,3031.60,2970.94,-0.0041,0.21,104.21,104.01,104.39,...,5611.85,5527.91,5627.56,5488.73,0.0055,17299.29,17045.44,17334.98,16854.37,-0.0014


In [55]:
data_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 762 entries, 0 to 761
Data columns (total 100 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   date                    762 non-null    datetime64[ns]
 1   imoex_price             762 non-null    float64       
 2   imoex_open              762 non-null    float64       
 3   imoex_high              762 non-null    float64       
 4   imoex_low               762 non-null    float64       
 5   imoex_change_percent    762 non-null    float64       
 6   cb_rate                 762 non-null    float64       
 7   dxy_price               756 non-null    float64       
 8   dxy_open                756 non-null    float64       
 9   dxy_high                756 non-null    float64       
 10  dxy_low                 756 non-null    float64       
 11  dxy_change_percent      756 non-null    float64       
 12  gold_price              751 non-null    float64  

In [56]:
# Удаление колонок, в названии которых содержится слово 'value'
EUR_RUB_df = EUR_RUB_df.loc[:, ~EUR_RUB_df.columns.str.contains('value', case=False)]

In [57]:
data_df = data_df.drop('moexit_value', axis=1)        
data_df = data_df.drop('moexre_value', axis=1) 

In [58]:
data_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 762 entries, 0 to 761
Data columns (total 98 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   date                    762 non-null    datetime64[ns]
 1   imoex_price             762 non-null    float64       
 2   imoex_open              762 non-null    float64       
 3   imoex_high              762 non-null    float64       
 4   imoex_low               762 non-null    float64       
 5   imoex_change_percent    762 non-null    float64       
 6   cb_rate                 762 non-null    float64       
 7   dxy_price               756 non-null    float64       
 8   dxy_open                756 non-null    float64       
 9   dxy_high                756 non-null    float64       
 10  dxy_low                 756 non-null    float64       
 11  dxy_change_percent      756 non-null    float64       
 12  gold_price              751 non-null    float64   

In [59]:
data_df.to_excel('data_df.xlsx', index=False)